# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
#found code here for opening zip files: https://stackoverflow.com/questions/26942476/reading-csv-zipped-files-in-python
import zipfile

orders = zipfile.ZipFile('./Orders.zip') 
orders = pd.read_csv(orders.open('Orders.csv'))
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [6]:
# your code here
# First sub problem: How to aggregate the amount_spent for unique customers
agg_sum_amount = orders.groupby(['CustomerID']).agg({'amount_spent': sum}).reset_index()
agg_sum_amount

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [15]:
# second sub problem: How to select customers whose aggregated amount_spent is in a given quantile range
#The first group is VIP Customers whose aggregated expenses at your global chain stores are above the 95th percentile 
#(aka. 0.95 quantile).
#The second group is Preferred Customers whose aggregated expenses are between the 75th and 95th percentile.

#Calculcating the 0.95 quantile for amount_spent
quant95_amount_spent = agg_sum_amount['amount_spent'].quantile(q=0.95)

#Calculcating the 0.75 quantile for amount_spent
quant75_amount_spent = agg_sum_amount['amount_spent'].quantile(q=0.75)

#Getting only customers with amount_spent between 75th and 95th percentile:
cust_75_95 = agg_sum_amount[(agg_sum_amount['amount_spent'] > quant75_amount_spent) & (agg_sum_amount['amount_spent'] < quant95_amount_spent)]

#Getting only customers with amount_spent above 95th percentile:
cust_over_95 = agg_sum_amount[agg_sum_amount['amount_spent'] >= quant95_amount_spent]
cust_over_95

,CustomerID,amount_spent
0,12346,77183.60
10,12357,6207.67
12,12359,6372.58
50,12409,11072.67
55,12415,124914.53
...,...,...
4207,18109,8052.97
4229,18139,8438.34
4253,18172,7561.68
4292,18223,6484.54


In [18]:
#third sub problem: How to label selected customers as "VIP" or "Preferred"
def labelCustomer(row):
    if row['amount_spent'] >=quant95_amount_spent:
        return 'VIP'
    elif (row['amount_spent'] > quant75_amount_spent) & (row['amount_spent'] < quant95_amount_spent):
        return 'Preferred'
    else:
        return 'normal'

In [19]:
#Applying function to create new column for labeling customers:
agg_sum_amount['Labeling'] = agg_sum_amount.apply(labelCustomer, axis=1)
agg_sum_amount.head()

,CustomerID,amount_spent,Labeling
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [22]:
# your code here
#Get subtable with CustomerID, Country and amount_spent
cust_count_amount = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent': sum}).reset_index()

#Apply labeling to the customers:
cust_count_amount['Labeling'] = cust_count_amount.apply(labelCustomer, axis=1)
cust_count_amount.head()

,CustomerID,Country,amount_spent,Labeling
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,normal


In [29]:
# How to identify which country has the most VIP Customers
#Intermediate: Group table by Country and Labeling and aggregate amount_spent sum and CustomerID counts for each
intermediate = cust_count_amount.groupby(['Country', 'Labeling']).agg({'amount_spent':sum, 'CustomerID':pd.Series.nunique}).reset_index()
intermediate

,Country,Labeling,amount_spent,CustomerID
0,Australia,Preferred,10056.290,3
1,Australia,VIP,124914.530,1
2,Australia,normal,3550.490,5
3,Austria,Preferred,2662.060,1
4,Austria,normal,7536.620,10
...,...,...,...,...
70,United Arab Emirates,normal,1902.280,2
71,United Kingdom,Preferred,2206713.761,755
72,United Kingdom,VIP,3417798.950,177
73,United Kingdom,normal,1683878.843,2989


In [34]:
#Filtering just for Labeling VIP
just_VIP = intermediate[intermediate['Labeling'] == 'VIP']
just_VIP

,Country,Labeling,amount_spent,CustomerID
1,Australia,VIP,124914.53,1
12,Channel Islands,VIP,8137.02,1
15,Cyprus,VIP,6372.58,1
19,Denmark,VIP,7829.89,1
22,EIRE,VIP,261204.69,2
25,Finland,VIP,7956.46,1
28,France,VIP,106383.09,9
31,Germany,VIP,100587.33,10
41,Japan,VIP,28406.43,2
47,Netherlands,VIP,280206.02,1


In [38]:
#Getting just the row for the country with the maximal amount of customers with VIP labeling
max_cust = just_VIP['CustomerID'].max()
just_VIP[just_VIP['CustomerID'] == max_cust]

,Country,Labeling,amount_spent,CustomerID
72,United Kingdom,VIP,3417798.95,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [41]:
# your code here
#Filter previous table to have Labeling column with VIP and Preferred
VIP_and_preferred = intermediate[(intermediate['Labeling'] == 'VIP') | (intermediate['Labeling'] == 'Preferred')]
VIP_and_preferred

,Country,Labeling,amount_spent,CustomerID
0,Australia,Preferred,10056.290,3
1,Australia,VIP,124914.530,1
3,Austria,Preferred,2662.060,1
6,Belgium,Preferred,31184.220,11
9,Canada,Preferred,2940.040,1
11,Channel Islands,Preferred,6048.510,3
12,Channel Islands,VIP,8137.020,1
14,Cyprus,Preferred,4967.390,2
15,Cyprus,VIP,6372.580,1
18,Denmark,Preferred,6727.720,2


In [43]:
#Group by the country to have the sum of customers per country 
combined = VIP_and_preferred.groupby(['Country']).agg({'amount_spent':sum, 'CustomerID': sum}).reset_index()
combined

,Country,amount_spent,CustomerID
0,Australia,134970.820,4
1,Austria,2662.060,1
2,Belgium,31184.220,11
3,Canada,2940.040,1
4,Channel Islands,14185.530,4
5,Cyprus,11339.970,3
6,Denmark,14557.610,3
7,EIRE,265545.900,3
8,Finland,18686.810,5
9,France,174694.670,29


In [44]:
# Get only the country with the most customers
max_cust_combined = combined['CustomerID'].max()
combined[combined['CustomerID'] == max_cust_combined]

,Country,amount_spent,CustomerID
26,United Kingdom,5624512.711,932
